In [2]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
# cfg = Config.fromfile('/opt/ml/detection/models/Swin/swinB_cascade.py')
cfg = Config.fromfile('/opt/ml/detection/level2-cv09/src/models/mmdetection/swinB_cascade_kfold.py')

root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
# cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
# cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]

# model 저장 위치 꼭 잘 확인하세요!
# 저장하고자하는 위치를 변경하세요.
cfg.work_dir = '/opt/ml/detection/baseline/mmdetection/work_dirs/swinB_cascade_3'

cfg.model.roi_head.bbox_head[0].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)












starts k-fold with /opt/ml/detection/dataset/cv_train3.json













In [4]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [5]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3918, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3158  | 1 [Paper]     | 5132  | 2 [Paper pack]  | 707   | 3 [Metal]   | 770   | 4 [Glass]    | 853   |
| 5 [Plastic]       | 2314  | 6 [Styrofoam] | 1004  | 7 [Plastic bag] | 4108  | 8 [Battery] | 129   | 9 [Clothing] | 375   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [6]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-03-25 03:48:33,336 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window7_224_22kto1k.pth
2022-03-25 03:48:33,716 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-03-25 03:48:33,743 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-03-25 03:48:33,754 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-03-25 03:48:33,863 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}},

Weight & Bias Project Init (1번만 실행하면 됩니다.)\
## config file에서 실험 이름(expr_name)을 꼭! 확인하세요!!

In [15]:
import wandb

# wandb.init(project="swinB-cascade-kfold", entity="cv-09")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: updaun (use `wandb login --relogin` to force relogin)


In [7]:
# 모델 학습

train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2022-03-25 03:50:13,113 - mmdet - INFO - Start running, host: root@81692cdfc99c, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/swinB_cascade_3
2022-03-25 03:50:13,115 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(LOW  

2022-03-25 03:50:29,363 - mmdet - INFO - Epoch [1][10/980]	lr: 6.350e-06, eta: 10:07:18, time: 0.930, data_time: 0.232, memory: 8248, loss_rpn_cls: 0.6796, loss_rpn_bbox: 0.0329, s0.loss_cls: 1.7237, s0.acc: 56.4404, s0.loss_bbox: 0.0133, s1.loss_cls: 0.5312, s1.acc: 81.6211, s1.loss_bbox: 0.0027, s2.loss_cls: 0.3935, s2.acc: 70.1953, s2.loss_bbox: 0.0003, loss: 3.3772, grad_norm: 45.8428
2022-03-25 03:50:35,818 - mmdet - INFO - Epoch [1][20/980]	lr: 7.850e-06, eta: 8:34:21, time: 0.646, data_time: 0.008, memory: 8248, loss_rpn_cls: 0.6252, loss_rpn_bbox: 0.0513, s0.loss_cls: 0.4977, s0.acc: 93.7988, s0.loss_bbox: 0.0768, s1.loss_cls: 0.1548, s1.acc: 97.9639, s1.loss_bbox: 0.0216, s2.loss_cls: 0.1051, s2.acc: 97.4658, s2.loss_bbox: 0.0035, loss: 1.5359, grad_norm: 14.2100
2022-03-25 03:50:42,321 - mmdet - INFO - Epoch [1][30/980]	lr: 9.350e-06, eta: 8:04:18, time: 0.650, data_time: 0.007, memory: 8248, loss_rpn_cls: 0.5533, loss_rpn_bbox: 0.0413, s0.loss_cls: 0.3042, s0.acc: 95.3857, s